# import libraries and setup environment

In [1]:
from openai import OpenAI 
import json 
from pprint import pprint
import time

In [2]:
# check open ai version 
import openai

print(openai.__version__)


1.3.7


In [3]:
# loads API keys from a local file
with open('api_keys.json', 'r') as file:
    config = json.load(file)

openai_api_key = config['OPENAI_API_KEY']

# Setup tooling 

In [131]:
tools = [{'type': 'retrieval'}]
tools

[{'type': 'retrieval'}]

In [132]:
# Custom function to Send an email
direction = {
    "name": "direction to move the robot",
    "description": "Moves the forward, backward, left or right",
    "parameters": {
        "type": "object",
        "properties": {
            "direction" : {
                "type": "string",
                "description" : "choose the direction to move the robot, the options are forward, backward, left or right"
            },
            "speed": {
                "type": "integer",
                "description" : "speed of the robot, the range is between 0 and 100, if not option is given, choose the default 50"

            },
        }
    },
    "required": ["direction", "speed"]
}


In [133]:

tools.append({'type': 'function', 'function': direction})

print(tools)

[{'type': 'retrieval'}, {'type': 'function', 'function': {'name': 'direction to move the robot', 'description': 'Moves the forward, backward, left or right', 'parameters': {'type': 'object', 'properties': {'direction': {'type': 'string', 'description': 'choose the direction to move the robot, the options are forward, backward, left or right'}, 'speed': {'type': 'integer', 'description': 'speed of the robot, the range is between 0 and 100, if not option is given, choose the default 50'}}}, 'required': ['direction', 'speed']}}]


# Create a robot agent (openai assistant)

In [134]:
assistant = client.beta.assistants.create(
    name="Robot_Agent",
    instructions="you area four wheeled robot that can move forward, backward, left, and right, you are awaiting instructions from the user.",
    tools = tools,
    model="gpt-4-1106-preview"
)
print(assistant)

Assistant(id='asst_ODrfXWXgtuPb7pUGpV2YaYhy', created_at=1701638878, description=None, file_ids=[], instructions='you area four wheeled robot that can move forward, backward, left, and right, you are awaiting instructions from the user.', metadata={}, model='gpt-4-1106-preview', name='Robot_Agent', object='assistant', tools=[ToolRetrieval(type='retrieval'), ToolFunction(function=FunctionDefinition(name='direction to move the robot', parameters={'type': 'object', 'properties': {'direction': {'type': 'string', 'description': 'choose the direction to move the robot, the options are forward, backward, left or right'}, 'speed': {'type': 'integer', 'description': 'speed of the robot, the range is between 0 and 100, if not option is given, choose the default 50'}}}, description='Moves the forward, backward, left or right'), type='function')])


In [135]:
#call this anytime you want to update an existing assistant
assistant = client.beta.assistants.update (
    "asst_ODrfXWXgtuPb7pUGpV2YaYhy",
    instructions="you area four wheeled robot that can move forward, backward, left, and right, you are awaiting instructions from the user.",
    tools=tools,
    model ="gpt-4-1106-preview")

print(assistant)

Assistant(id='asst_ODrfXWXgtuPb7pUGpV2YaYhy', created_at=1701638878, description=None, file_ids=[], instructions='you area four wheeled robot that can move forward, backward, left, and right, you are awaiting instructions from the user.', metadata={}, model='gpt-4-1106-preview', name='Robot_Agent', object='assistant', tools=[ToolRetrieval(type='retrieval'), ToolFunction(function=FunctionDefinition(name='direction to move the robot', parameters={'type': 'object', 'properties': {'direction': {'type': 'string', 'description': 'choose the direction to move the robot, the options are forward, backward, left or right'}, 'speed': {'type': 'integer', 'description': 'speed of the robot, the range is between 0 and 100, if not option is given, choose the default 50'}}}, description='Moves the forward, backward, left or right'), type='function')])


In [136]:
#to retrieve all your defined assistants
my_assistants = client.beta.assistants.list(
    order="desc",
    limit = "20"
)

pprint(my_assistants.data)

[Assistant(id='asst_ODrfXWXgtuPb7pUGpV2YaYhy', created_at=1701638878, description=None, file_ids=[], instructions='you area four wheeled robot that can move forward, backward, left, and right, you are awaiting instructions from the user.', metadata={}, model='gpt-4-1106-preview', name='Robot_Agent', object='assistant', tools=[ToolRetrieval(type='retrieval'), ToolFunction(function=FunctionDefinition(name='direction to move the robot', parameters={'type': 'object', 'properties': {'direction': {'type': 'string', 'description': 'choose the direction to move the robot, the options are forward, backward, left or right'}, 'speed': {'type': 'integer', 'description': 'speed of the robot, the range is between 0 and 100, if not option is given, choose the default 50'}}}, description='Moves the forward, backward, left or right'), type='function')]),
 Assistant(id='asst_qp1ossSjXhWFJUJQwXY264TF', created_at=1701638761, description=None, file_ids=[], instructions='you area four wheeled robot that ca

# Create a the thread for the robot agent

A Thread represents a conversation session between the assistant and a user. It stores Messages, providing context for ongoing interactions.

In [137]:
# Creates a new conversation
thread = client.beta.threads.create()

In [138]:
# Create a new message in the thread as the user (role) 
# this doesnt run the model yet
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content='please move forward, then backward, then turn left, then turn right'
)
print(message)

ThreadMessage(id='msg_QFHVcWVFFoTWqpqSf2CWjkS4', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='please move forward, then backward, then turn left, then turn right'), type='text')], created_at=1701638898, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_ytX9xRNmV4NXL9PsYKNyES81')


In [139]:
# this runs the model with the input from thread 
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
)

print(run.id)

run_4HDjg9pUw8EUcjAFAqk1WUuG


In [140]:
# This retrieves the output from the model
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

print(run.status)

in_progress


In [141]:
# This checks the run status and waits until the run is completed or has failed or requires action
# and then prints the status.  Basically it waits until the model has finished running
while run.status not in ["completed", "failed", "requires_action"]:
    run = client.beta.threads.runs.retrieve(
thread_id = thread.id,
run_id = run.id
)

print(run.status)
time.sleep(10)
print(run.status)

requires_action
requires_action


In [142]:
# This retrieves the output from the model
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
)

for each in messages:
  pprint(each.role + ":" + each.content[0].text.value)

'user:please move forward, then backward, then turn left, then turn right'


# Extract the tool from the robot agent 

In [143]:
# This runs the model with the input from thread
tools_to_call = run.required_action.submit_tool_outputs.tool_calls
print(len(tools_to_call))
print(tools_to_call)

4
[RequiredActionFunctionToolCall(id='call_xEOyyx74vxUK29Cgw86D5AxV', function=Function(arguments='{"direction": "forward"}', name='direction to move the robot'), type='function'), RequiredActionFunctionToolCall(id='call_BcA4QAr1FgK0lLaxNn4lj5pG', function=Function(arguments='{"direction": "backward"}', name='direction to move the robot'), type='function'), RequiredActionFunctionToolCall(id='call_DMhzogNFpRbJl0Ycp8hDH25u', function=Function(arguments='{"direction": "left"}', name='direction to move the robot'), type='function'), RequiredActionFunctionToolCall(id='call_VUc6mSQw1AaIqE0Fp9Kps1KO', function=Function(arguments='{"direction": "right"}', name='direction to move the robot'), type='function')]


In [144]:
# prints the 1st tool call name and arguments (there could be more than one tool call so this wont work for all)
print(tools_to_call[0].function.name)
print(tools_to_call[0].function.arguments)

direction to move the robot
{"direction": "forward"}


In [145]:
# this prints for each tool call the name and arguments
tools_output_array = []
for each_tool in tools_to_call:
  tool_call_id = each_tool.id
  function_name = each_tool.function.name
  function_arg = each_tool.function.arguments
  print("Tool ID:" + tool_call_id)
  print("Function to Call:" + function_name )
  print("Parameters to use:" + function_arg)

  if (function_name == 'move_forward'):
    output=True

  tools_output_array.append({"tool_call_id": tool_call_id, "output": output})

print(tools_output_array)

Tool ID:call_xEOyyx74vxUK29Cgw86D5AxV
Function to Call:direction to move the robot
Parameters to use:{"direction": "forward"}
Tool ID:call_BcA4QAr1FgK0lLaxNn4lj5pG
Function to Call:direction to move the robot
Parameters to use:{"direction": "backward"}
Tool ID:call_DMhzogNFpRbJl0Ycp8hDH25u
Function to Call:direction to move the robot
Parameters to use:{"direction": "left"}
Tool ID:call_VUc6mSQw1AaIqE0Fp9Kps1KO
Function to Call:direction to move the robot
Parameters to use:{"direction": "right"}
[{'tool_call_id': 'call_xEOyyx74vxUK29Cgw86D5AxV', 'output': True}, {'tool_call_id': 'call_BcA4QAr1FgK0lLaxNn4lj5pG', 'output': True}, {'tool_call_id': 'call_DMhzogNFpRbJl0Ycp8hDH25u', 'output': True}, {'tool_call_id': 'call_VUc6mSQw1AaIqE0Fp9Kps1KO', 'output': True}]


# Use the tool and perform a action 

In [146]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_calls

[RequiredActionFunctionToolCall(id='call_xEOyyx74vxUK29Cgw86D5AxV', function=Function(arguments='{"direction": "forward"}', name='direction to move the robot'), type='function'),
 RequiredActionFunctionToolCall(id='call_BcA4QAr1FgK0lLaxNn4lj5pG', function=Function(arguments='{"direction": "backward"}', name='direction to move the robot'), type='function'),
 RequiredActionFunctionToolCall(id='call_DMhzogNFpRbJl0Ycp8hDH25u', function=Function(arguments='{"direction": "left"}', name='direction to move the robot'), type='function'),
 RequiredActionFunctionToolCall(id='call_VUc6mSQw1AaIqE0Fp9Kps1KO', function=Function(arguments='{"direction": "right"}', name='direction to move the robot'), type='function')]

In [148]:
direction_to_move = json.loads(tool_calls[0].function.arguments)["direction"]
#speed_to_move = json.loads(tool_calls[0].function.arguments)["speed"]


In [149]:
def direction_to_move_robot(direction, speed=50):
    return f"move {direction} at speed {speed}"

In [150]:
import json

def get_outputs_for_tool_call(tool_call):
    # calls apis, would need to change to call the function
    direction_to_move = json.loads(tool_calls[0].function.arguments)["direction"]
    #speed_to_move = json.loads(tool_calls[0].function.arguments)["speed"]
    robot_status = direction_to_move_robot(direction_to_move)
    return {
        "tool_call_id": tool_call.id,
        "output": robot_status
        }
tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_outputs = map(get_outputs_for_tool_call, tool_calls)


In [151]:
tool_outputs = list(tool_outputs)


In [152]:
tool_outputs

[{'tool_call_id': 'call_xEOyyx74vxUK29Cgw86D5AxV',
  'output': 'move forward at speed 50'},
 {'tool_call_id': 'call_BcA4QAr1FgK0lLaxNn4lj5pG',
  'output': 'move forward at speed 50'},
 {'tool_call_id': 'call_DMhzogNFpRbJl0Ycp8hDH25u',
  'output': 'move forward at speed 50'},
 {'tool_call_id': 'call_VUc6mSQw1AaIqE0Fp9Kps1KO',
  'output': 'move forward at speed 50'}]

In [153]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id = run.id,
    tool_outputs = tool_outputs
)

In [154]:
# This retrieves the output from the model
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
)

for each in messages:
  pprint(each.role + ":" + each.content[0].text.value)

('assistant:The instructions have been executed; I have moved forward, then '
 'backward, turned left, and then right.')
'user:please move forward, then backward, then turn left, then turn right'


# removes the agent (assistant)

In [117]:
# remove assistant 
response = client.beta.assistants.delete(assistant.id)
print(response)

AssistantDeleted(id='asst_teGR4Px4tcsvHqYi5vqMhDjh', deleted=True, object='assistant.deleted')
